In [1]:
!pip install -r requirements.txt

  Using cached pyspark-3.4.4.tar.gz (311.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl (11.6 MB)
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: finished with status 'done'


  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
import os 
os.environ['JAVA_HOME'] = r'C:\tools\jdk-11.0.25'
os.environ['HADOOP_HOME'] = r'C:\tools\hadoop-3.2.0'
os.environ['PATH'] = f"{os.environ['HADOOP_HOME']}\\bin;{os.environ['PATH']}"  # Add Hadoop to PATH
os.environ['SPARK_HOME'] = r'C:\tools\spark-3.4.4-bin-hadoop3'
os.environ['PYSPARK_PYTHON'] = 'python'

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!curl -L -o yellow_tripdata_2024-10.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 16 61.3M   16  9.9M    0     0  9525k      0  0:00:06  0:00:01  0:00:05 9525k
 34 61.3M   34 21.2M    0     0  10.2M      0  0:00:05  0:00:02  0:00:03 10.2M
 53 61.3M   53 32.5M    0     0  10.5M      0  0:00:05  0:00:03  0:00:02 10.5M
 71 61.3M   71 43.8M    0     0  10.7M      0  0:00:05  0:00:04  0:00:01 10.7M
 89 61.3M   89 55.0M    0     0  10.8M      0  0:00:05  0:00:05 --:--:-- 11.0M
100 61.3M  100 61.3M    0     0  10.8M      0  0:00:05  0:00:05 --:--:-- 11.2M


In [6]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [7]:
df = df.repartition(4)

In [8]:
df.write.parquet('data/yellow/2024/10/')

In [20]:
from datetime import datetime 
from pyspark.sql.functions import to_date
oct15 = datetime(2024,10,15)

oct15_df = df.filter(to_date(df.tpep_pickup_datetime) == oct15)
oct15_df.count()

128893

In [23]:
df.registerTempTable("trips")

c:\Users\Kenta Sakai\projects\data-engineering-zoomcamp\cohorts\2025\05-batch\venv\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [ ]:
spark.sql("""
          SELECT 
          (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 as hours,
          tpep_dropoff_datetime,
          tpep_pickup_datetime
          FROM trips
          order by 1 desc
""").show()

In [ ]:
!curl -o data/taxi_zone_lookup.csv https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12331  100 12331    0     0  96278      0 --:--:-- --:--:-- --:--:-- 96335


In [37]:
zones = spark.read \
    .option("header", "true") \
    .csv("data/taxi_zone_lookup.csv")

In [38]:
zones.registerTempTable("zones")

c:\Users\Kenta Sakai\projects\data-engineering-zoomcamp\cohorts\2025\05-batch\venv\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [40]:
zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [42]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [49]:
spark.sql(
    """
    WITH least AS (
        SELECT PULocationID, COUNT(*)
        FROM trips t 
        GROUP BY 1 
        ORDER BY 2 ASC
        LIMIT 1
    )
    
    SELECT Zone FROM zones WHERE LocationID in (SELECT PULocationID FROM least )
""").show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
+--------------------+

